## Application - Pressure Poisson Equation

The momentum equation for the velocity field $\vec{u}$ in a fluid is

\begin{equation}
\frac{\partial \vec{u}}{\partial t}+(\vec{u}\cdot\nabla)\vec{u}=-\frac{1}{\rho}\nabla p + \nu \nabla^2\vec{u}
\end{equation}

where $p$ is the pressure, $\nu$ is the fluid viscosity and $\rho$ is the fluid density. With three velocity components, plus the pressure, we have four unknowns but only three equations. For compressible fluids, we have an equation of state to complete the system. In the incompressible case, we don't have an equation of state and we need an additional constraint from somewhere else. 

This is what we do: take the divergence of the momentum equation, apply the incompressibility constraint to cancel some terms, and get an equation for the pressure. It's a pretty cool trick.

Conservation of mass for an incompressible fluid requires that the divergence of $\vec{u}$ must be zero:

$$\nabla \cdot \vec{u} = 0$$

Writing out the momentum equation in $x$ and $y$ (for 2d flow), we get

$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial x}+\nu \left(\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right) $$


$$\frac{\partial v}{\partial t}+u\frac{\partial v}{\partial x}+v\frac{\partial v}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial y}+\nu\left(\frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2}\right) $$

We take the divergence of the momentum equation and then apply the incompressibility constraint.  After some wrangling and cancellations, this leaves us with the pressure poisson equation.

$$\frac{\partial^2 p}{\partial x^2}+\frac{\partial^2 p}{\partial y^2} = -\rho\left(\frac{\partial u}{\partial x}\frac{\partial u}{\partial x}+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\frac{\partial v}{\partial y}\frac{\partial v}{\partial y} \right)$$

Which is of the form

$$\frac{\partial ^2 p}{\partial x^2} + \frac{\partial ^2 p}{\partial y^2} = b$$

Imagine we discretize a domain as in the figure below:

<img src='./figures/2dgrid.svg' width=350>

Then the left-hand side of the equation 

$$\frac{\partial ^2 p}{\partial x^2} + \frac{\partial ^2 p}{\partial y^2} = b$$

discretized using 2nd-order central differences would read


$$\frac{p^n_{i+1, j} - 2p^n_{i,j} + p^n_{i - 1, j}}{\Delta x ^2} + \frac{p^n_{i, j+1} - 2p^n_{i,j} + p^n_{i, j-1}}{\Delta y ^2}$$



where subscripts denote position within a cartesian coordinate system and superscripts denote position in time.

We apply a similar discretization to the momentum equation (forward-time, backward-space for the 1st-order terms) and also assume a uniform domain, so $\Delta x = \Delta y$


## Solution procedure

### Initial velocity field

We start with a velocity field in $u$ and $v$ at some timestep `n`. 

### Calculate pressure

Then, we iteratively solve the Poisson equation for pressure, where the $k$ superscript denotes an iteration in 'pseudo-time' (we repeatedly solve the Poisson equation until it reaches a quasi-steady state):


\begin{align}
p_{i,j}^{k+1} &= \frac{1}{4}\left(p_{i+1,j}^{k}+p_{i-1,j}^{k}+p_{i,j+1}^{k}+p_{i,j-1}^{k}\right) \\
&-\frac{\rho \Delta x}{16} \left( \frac{2}{\Delta t} \left(u_{i+1,j} - u_{i-1,j} + v_{i,j+1} - v_{i,j-1}\right) \right . \\
&-\frac{2}{\Delta x}\left(u_{i,j+1} - u_{i,j-1} \right) \left(v_{i+1,j} - v_{i-1,j} \right) \\
&- \left . \frac{\left(u_{i+1,j} - u_{i-1,j} \right)^2}{\Delta x} 
- \frac{ \left(v_{i,j+1} - v_{i,j-1} \right)^2 }{\Delta x} \right) \\
\end{align}

### Update the velocity

Once the Poisson equation reaches this quasi-steady state, we take the pressure field as $p^n$ and then use it to solve for the velocity in $u$ and $v$ at the next timestep, $n+1$.

The momentum equation in the $u$ direction:

\begin{align}
u_{i,j}^{n+1} = u_{i,j}^{n} &- \frac{\Delta t}{\Delta x} \left( u_{i,j}^{n}(u_{i,j}^{n}-u_{i-1,j}^{n})
+ v_{i,j}^{n} (u_{i,j}^{n}-u_{i,j-1}^{n}) + \frac{1}{2 \rho}(p_{i+1,j}^{n}-p_{i-1,j}^{n}) \right) \\
&+\frac{\nu \Delta t}{\Delta x^2}\left(u_{i+1,j}^{n} + u_{i-1,j}^{n} + u_{i,j+1}^{n} + u_{i,j-1}^{n} -4u_{i,j}^{n}\right)
\end{align}

The momentum equation in the $v$ direction:

\begin{align}
v_{i,j}^{n+1} = v_{i,j}^{n} &- \frac{\Delta t}{\Delta x} \left( u_{i,j}^{n}(v_{i,j}^{n}-v_{i-1,j}^{n})
+ v_{i,j}^{n} (v_{i,j}^{n}-v_{i,j-1}^{n}) + \frac{1}{2 \rho}(p_{i,j+1}^{n}-p_{i,j-1}^{n}) \right) \\
&+\frac{\nu \Delta t}{\Delta x^2}\left(v_{i+1,j}^{n} + v_{i-1,j}^{n} + v_{i,j+1}^{n} + v_{i,j-1}^{n} -4v_{i,j}^{n}\right)
\end{align}



Then, rinse and repeat.